In [1]:
#using Pkg
#Pkg.add(["JuMP", "StatsPlots","Clp","HiGHS"])
using Distributions, Random, PrettyTables, QuadGK, Plots, Roots, JLD2, Optim, Plots, Clp, StatsPlots, JuMP, HiGHS, LinearAlgebra

### (b).

In [2]:
#DGP 
function dgp(n,π1,ρ,c)
     
    u= rand(Uniform(0,1),n)
    e= randn(n,1)
    v= ρ*u+e
    
    Z= randn(n,1)
    α=c*u
    
    D= ones(n,1)+Z.*π+v
    Y= ones(n,1)+α.*D
       
   return Y, D, Z
    
end

dgp (generic function with 1 method)

### (c).

In [ ]:
function qreg(;Y,X,τ)
    
    QR=Model(HiGHS.Optimizer)
    set_silent(QR)
    n=length(Y)
    
    c=[τ*ones(n,1);(1-τ)*ones(n,1);];
    @variable(QR,x[1:2*n]>=0.0); #this includes the non-negativity constraint on z^{+} and z^{-}
    @variable(QR,-100.0<=b<=100.0); #if you want to estimate vector b you use @variable(QR,-100.0<=b[1:k]<=100.0) where k is dimension
    @objective(QR,Min,sum(c[i]*x[i] for i in 1:(2*n)));
    @constraint(QR,constraint[i in 1:n],x[i]-x[n+i]+sum(X[i,:].*b)+sum(Z[i,:].*g)==Y[i]); #the equality constraint
    optimize!(QR);
    estr=value.(b)
    
end